# Cebuano Stemmer Demo

`Accepts a Cebuano word and returns the root word and its affixes`

In [1]:
from stemmer import stem_word

ceb_word = raw_input('Input a Cebuano word: ')
word = stem_word(word=ceb_word)

print(word.print_stem_results())

Input a Cebuano word: niadtong
niadtong:[(adto), {ni,-,ng}]


# Cebuano POS Tagger Demo

`Accepts a Cebuano sentence and return POS tagged sentence`

In [2]:
from tagger import tag_sentence

ceb_sentence = raw_input('Input a Cebuano sentence: ')
words = tag_sentence(text=ceb_sentence)

sentence = ''
for word in words:
    sentence += str(word) + ' '

print(sentence)

Input a Cebuano sentence: Kini base usab sa natala nga mga bus trips nga niabot og 592 niadtong Biyernes, 579 niadtong Sabado, 655 niadtong Domingo ug 556 niadtong Lunes, adlaw sa Pasko.
kini/['PRON'] base/['NOUN'] usab/['VERB'] sa/['PART'] natala/['VERB', 'NOUN'] nga/['PART'] mga/['PART'] bus/['NOUN'] trips/['OTH'] nga/['PART'] niabot/['VERB', 'NOUN'] og/['CONJ', 'DET'] 592/['NUM'] niadtong/['PRON', 'VERB'] biyernes/['NOUN'] ,/['SYM'] 579/['NUM'] niadtong/['PRON', 'VERB'] sabado/['NOUN'] ,/['SYM'] 655/['NUM'] niadtong/['PRON', 'VERB'] domingo/['NOUN'] ug/['CONJ', 'DET'] 556/['NUM'] niadtong/['PRON', 'VERB'] lunes/['NOUN'] ,/['SYM'] adlaw/['NOUN'] sa/['PART'] pasko/['NOUN'] ./['SYM'] 


# Stemmer Evaluation


`Evaluates the stemmer by getting the number of predicted root words vs. actual root words`

* The input words are already tokenized and stemmed

In [1]:
import stemmer_evaluator as se
import pandas as pd

result = se.to_panda_data()

df = pd.DataFrame(result['data'], index=result['index'])
df_valid = df[(df.is_valid == True)]
valid = df_valid.count()['is_valid']
print('Total tokens: ' + str(df.count()['is_valid']))
print('Correct root words: ' + str(df[(df.is_valid == True)].count()['is_valid']))
print('Incorrect root words: ' + str(df[(df.is_valid == False)].count()['is_valid']))
print("Correct / incorrect root percentage: " + str((valid / float(df.count()['is_valid'])) * 100))
print('\n')
print('Dictionary entry tokens: ' + str(df[(df.is_entry == True)].count()['is_entry']))
print('Correct dictionary entry tokens: ' + str(df[(df.is_entry == True) & (df.is_valid == True)].count()['is_entry']))
print('Incorrect dictionary entry tokens: ' + str(df[(df.is_entry == True) & (df.is_valid == False)].count()['is_entry']))
print('\n')
print('Non-dictionary entry tokens: ' + str(df[(df.is_entry == False)].count()['is_entry']))
print('Correct non-dictionary entry tokens: ' + str(df[(df.is_entry == False) & (df.is_valid == True)].count()['is_entry']))
print('Incorrect non-dictionary entry tokens: ' + str(df[(df.is_entry == False) & (df.is_valid == False)].count()['is_entry']))
print('\n')

df.tail(n=10)


Total tokens: 1298
Correct root words: 955
Incorrect root words: 343
Correct / incorrect root percentage: 73.57473035439138


Dictionary entry tokens: 974
Correct dictionary entry tokens: 878
Incorrect dictionary entry tokens: 96


Non-dictionary entry tokens: 324
Correct non-dictionary entry tokens: 77
Incorrect non-dictionary entry tokens: 247




,infix,is_entry,is_root,is_valid,prefix,root,suffix
nipahibawo,NaN,False,False,False,ni,pahibaw,o
naghuwat,NaN,True,False,True,nag,huwat,NaN
unsaon,NaN,True,False,True,NaN,unsa,on
mamahimong,NaN,True,False,True,ma,himo,ng
syudad,NaN,True,True,True,NaN,syudad,NaN
sirhan,NaN,True,False,False,NaN,sir,han
kalapasan,NaN,True,False,True,ka,lapas,an
bawian,NaN,False,False,False,ba,wi,an
nakalapas,NaN,True,False,True,naka,lapas,NaN
nagkadaiyang,NaN,True,False,False,nag,kadaiya,ng


# Tagger Evaluation


`Evaluates the tagger by getting the number of predicted pos tags vs. actual pos tags (F-score)`

In [5]:
from pandas_ml import ConfusionMatrix
from sklearn.metrics import f1_score
import tagger_evaluator as te

words = te.tag_test_sentences()
y_true = te.extract_actual_pos_tags()
y_pred = te.extract_predicted_pos_tags(words=words)
confusion_matrix = ConfusionMatrix(y_true, y_pred)
confusion_matrix

Number of tokens: 1220

Disambiguated: 81.8852459016% (999) 



Predicted  ADJ  ADV  CONJ  DET  NOUN  NUM  OTH  PART  PRON  SYM  VERB  __all__
Actual                                                                        
ADJ         18   10     0    0     5    0    3     1     9    0     7       53
ADV          0   90     0    0     1    0    1     0     0    0     0       92
CONJ         0    0    52    0     0    0    0     0     0    0     0       52
DET          0    0    11   70     0    0    2     0     0    0     0       83
NOUN        12    0     1    1   226    0   50     1     0    0    23      314
NUM          0    8     0    0     1   28    1     0     0    0     0       38
OTH          0    0     0    0     0    0    3     0     0    0     0        3
PART         0    1     2    1     0    0    1   276    17    0     5      303
PRON         0    1     0    0     0    0    1     0    50    0     0       52
SYM          0    0     0    0     0    0    0     0     0   87     0       87
VERB         5    5     0    0     3    0   26     0

In [4]:
f1_score(y_true, y_pred, average=None)


array([0.40909091, 0.86956522, 0.88135593, 0.90322581, 0.82181818,
       0.84848485, 0.06593407, 0.95008606, 0.74626866, 1.        ,
       0.71014493])